## Imports

In [ ]:
import cv2
import json
import math
import numpy as np
from pathlib import Path



## Constants

In [62]:
feature_location_json = Path('../config/FeatureLocation.json')
chip_location_json = Path('../Label_templates/Chip_map_list.json')

image_path = Path('/Volumes/krauss/Lisa/GMR/Array/250325/loc1_1/Pos0/img_000000000_Default_000.tif')

b_left_template_path = Path('../Label_templates/IMECII/IMECII_2/B-Left.png')
e_left_template_path = Path('../Label_templates/IMECII/IMECII_2/E-Left.png')

## Outline

1. ☑ Load user feature location JSON file

2. ☑ Calculate the scale factor and image angle using the user feature location

3. ☑ Load the chip map JSON file and extract the labels location

4. ☑ Calculate the scale factor and image angle using the chip map

5. ☑ Rotate the image and user feature locations using calculated angle

6. ☑ Load template image and scale according to the scale factor

7. ☑ Find the location of the template in the image using cv2.matchTemplate

8. ☐ Display the results of the template matching on the image

9. ☐ Calculate some FOM values based on the template matching results

10. ☐ If matching FOM is below some threshold, attempt to match with a different technigue

11. ☐ Comparem matching results, if one is better, use that one

12. ☐ Calculate chip map transformation to match image

13. ☐ Save list of ROIs (i.e. grating locations) to a JSON file using transformation

☐ Unchecked
☑ Checked

## Load user feature locations from JSON file and populate 'user_chip_mapping' dictionary

In [ ]:
def load_json(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{file_path}'. Check the file format.")
        return None
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
        return None


In [67]:
def load_user_feature_locations(file_path):
    user_raw_data = load_json(file_path)
    user_chip_mapping = {}
    if user_raw_data:
        user_chip_mapping['chip_type'] = user_raw_data.get('chip_type', None)
        features = []
        for feature in user_raw_data.get('features', []):
            label_name = feature.get('label')
            feature_location = feature.get('feature_location')
            if label_name:
                features.append({
                    'label': label_name,
                    'user_location': feature_location
                })
        user_chip_mapping['features'] = features
    else:
        print("No valid data to load.")
    return user_chip_mapping

user_chip_mapping = load_user_feature_locations(feature_location_json)

if user_chip_mapping:
    print('User data loaded successfully:')
    print(user_chip_mapping)


User data loaded successfully:
{'chip_type': 'IMECII_2', 'features': [{'label': 'D-Right', 'user_location': [1840, 849]}, {'label': 'E-Right', 'user_location': [1848, 122]}]}


## Load chip map locations from JSON and update 'user_chip_map' dictionary

In [68]:
def get_type_of_chip(chip_type, all_chip_mappings):
    for chip_mapping in all_chip_mappings:
        chip_name_label = chip_mapping.get('chip_type', None)
        if chip_name_label == chip_type:
            return chip_mapping
    return None

def get_location_from_label(label, chip_mapping):
    for chip_label in chip_mapping.get('labels'):
        if chip_label.get('label') == label:
            return chip_label.get('label_origin', None)
    return None

def get_user_label_locations_from_chip_map(chip_mapping, user_chip_mapping):
    user_features = user_chip_mapping.get('features', None)
    if user_features:
        for idx, f in enumerate(user_features):
            label = f.get('label')
            feature_location = get_location_from_label(label, chip_mapping)
            user_chip_mapping['features'][idx]['chip_location'] = feature_location
    return user_chip_mapping

def load_chip_feature_locations(file_path, user_chip_mapping):
    chip_raw_data = load_json(file_path)
    chip_type = user_chip_mapping.get('chip_type')
    chip_mapping = get_type_of_chip(chip_type, chip_raw_data)
    return get_user_label_locations_from_chip_map(chip_mapping, user_chip_mapping)

user_chip_mapping = load_chip_feature_locations(chip_location_json, user_chip_mapping)
if user_chip_mapping:
    print('User data updated successfully:')
    print(user_chip_mapping)

User data updated successfully:
{'chip_type': 'IMECII_2', 'features': [{'label': 'D-Right', 'user_location': [1840, 849], 'chip_location': [2261, -1040]}, {'label': 'E-Right', 'user_location': [1848, 122], 'chip_location': [2261, -2040]}]}


## Calculate the scale factor and image angle using the user feature location

In [99]:
def angle_between_points(v1, v2):
    """Calculates the signed angle in degrees between the line connecting two vectors
    and the positive x-axis.

    Args:
        v1: (x1, y1)
        v2: (x2, y2)

    Returns:
        Angle in degrees, positive for counter-clockwise rotation from the
        positive x-axis to the line segment from point1 to point2.
    """
    x1, y1 = v1
    x2, y2 = v2

    dx = x2 - x1
    dy = y2 - y1

    return math.degrees(math.atan2(dy, dx))  # Angle relative to positive x-axis


def user_chip_rotation_angle(user_chip_mapping):
  image_f1 = user_chip_mapping['features'][0]['user_location']
  image_f2 = user_chip_mapping['features'][1]['user_location']
  image_angle = angle_between_points(image_f1, image_f2)

  chip_f1 = user_chip_mapping['features'][0]['chip_location']
  chip_f2 = user_chip_mapping['features'][1]['chip_location']
  map_angle = angle_between_points(chip_f1, chip_f2)

  rotation_angle = map_angle - image_angle

  user_chip_mapping['user_location_angle'] = image_angle
  user_chip_mapping['chip_location_angle'] = map_angle
  user_chip_mapping['rotation_angle'] = rotation_angle

  # print(f'Angle between user features: {image_angle:.2f} degrees')
  # print(f'Angle between map features: {map_angle:.2f} degrees')
  # print(f'Rotation angle: {rotation_angle:.2f} degrees')
  return user_chip_mapping, rotation_angle


user_chip_mapping, _ = user_chip_rotation_angle(user_chip_mapping)
if user_chip_mapping:
    print('User data updated successfully:')
    print(user_chip_mapping)


User data updated successfully:
{'chip_type': 'IMECII_2', 'features': [{'label': 'D-Right', 'user_location': [1840, 849], 'chip_location': [2261, -1040], 'refined_location': [1822, 825]}, {'label': 'E-Right', 'user_location': [1848, 122], 'chip_location': [2261, -2040], 'refined_location': [1817, 99]}], 'rotation_angle': -0.6304645613991369, 'scale_factor': 0.7270440151737719, 'user_location_angle': -89.36953543860086, 'chip_location_angle': -90.0}


In [93]:
def calculate_distance(v1, v2):
    """Calculates the Euclidean distance between two points in 2D space.

    Args:
      v1: A tuple or list representing the first point (x1, y1).
      v1: A tuple or list representing the second point (x2, y2).

    Returns:
      The Euclidean distance between the two points.
    """
    x1, y1 = v1
    x2, y2 = v2

    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def user_chip_scale_factor(user_chip_mapping):
    f1 = user_chip_mapping['features'][0]['user_location']
    f2 = user_chip_mapping['features'][1]['user_location']
    user_distance = calculate_distance(f1, f2)

    f1 = user_chip_mapping['features'][0]['chip_location']
    f2 = user_chip_mapping['features'][1]['chip_location']
    chip_distance = calculate_distance(f1, f2)

    scale_factor = user_distance / chip_distance

    user_chip_mapping['scale_factor'] = scale_factor

    # print(f'Distance between user features: {user_distance:.2f} pixels')
    # print(f'Distance between map features: {chip_distance:.2f} microns')
    # print(f'Scale factor: {scale_factor:.2f} pixels/micron')
    return user_chip_mapping, scale_factor

user_chip_mapping, _ = user_chip_scale_factor(user_chip_mapping)
if user_chip_mapping:
    print('User data updated successfully:')
    print(user_chip_mapping)


User data updated successfully:
{'chip_type': 'IMECII_2', 'features': [{'label': 'D-Right', 'user_location': [1840, 849], 'chip_location': [2261, -1040]}, {'label': 'E-Right', 'user_location': [1848, 122], 'chip_location': [2261, -2040]}], 'rotation_angle': -0.6304645613991369, 'scale_factor': 0.7270440151737719}


## Rotate the image and user feature locations using calculated angle

In [86]:
def rotate_image(image, rotation_angle):
    """
    Rotates an image based on corresponding feature points.

    Args:
        image: The input image (OpenCV format).
        rotation_angle: rotation angle

    Returns:
        rotated_image: The rotated image
    """

    # Calculate the center of rotation (midpoint of the image points)
    h, w = image.shape
    center = (w / 2, h / 2)

    # Get the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)

    # Apply the rotation and scaling
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))

    return rotated_image


def rotate_user_feature_locations(user_location, image_center, rotation_angle):
    """Rotates a point around a center by a given angle in degrees.

    Args:
        user_location: The point to rotate (x, y).
        image_center: The center of rotation (x, y).
        rotation_angle: rotation angle

    Returns:
        The rotated point (x, y).
    """
    x, y = user_location
    cx, cy = image_center
    angle_rad = math.radians(rotation_angle)

    rotated_x = cx + (x - cx) * math.cos(angle_rad) - (y - cy) * math.sin(angle_rad)
    rotated_y = cy + (x - cx) * math.sin(angle_rad) + (y - cy) * math.cos(angle_rad)
    return [int(rotated_x), int(rotated_y)]

## Load template image and scale according to the scale factor

In [72]:
def load_template(template_path):
    """
    Loads a template image from the specified path.

    Args:
        template_path: Path to the template image.

    Returns:
        The loaded template image.
    """
    try:
        template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
        if template is None:
            raise ValueError(f"Could not load image at {template_path}")
        return template
    except Exception as e:
        print(f"Error loading template: {e}")
        return None

b_left_template = load_template(b_left_template_path)
e_left_template = load_template(e_left_template_path)

print(f'Shape of B-Left template: {b_left_template.shape}')
print(f'Shape of E-Left template: {e_left_template.shape}')

Shape of B-Left template: (80, 57)
Shape of E-Left template: (80, 57)


In [80]:
def scale_template(template, scale_factor):
    """Scales a template image by a given factor.

    Args:
        template: The template image (OpenCV format).
        scale_factor: The scaling factor.

    Returns:
        The scaled template image.
    """
    new_size = (int(template.shape[1] * scale_factor), int(template.shape[0] * scale_factor))
    scaled_template = cv2.resize(template, new_size, interpolation=cv2.INTER_LINEAR)
    return scaled_template

scaled_b_left_template = scale_template(b_left_template, 0.5)
scaled_e_left_template = scale_template(e_left_template, 0.5)

print(f'Scaled shape of B-Left template: {scaled_b_left_template.shape}')
print(f'Scaled shape of E-Left template: {scaled_e_left_template.shape}')

Scaled shape of B-Left template: (40, 28)
Scaled shape of E-Left template: (40, 28)


## Find the location of the template in the image using cv2.matchTemplate

In [103]:
def get_template_image_from_label(chip_type, label):
    template_path = Path(f'../Label_templates/IMECII/{chip_type}/{label}.png')
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    if template is None:
        print(f"Error: Could not load template image {template_path}.")
        return None
    return template


def refine_feature_locations(image, user_chip_mapping):
    """Refines the location of a feature using template matching,
    accounting for image rotation.

    Args:
        image: The image to search in (already rotated).
        initial_location: The initial (approximate) location of the feature (x, y) in the *original* image coordinates.
        template: The template image of the feature.
        rotation_angle: The angle by which the image was rotated (in degrees, clockwise).
        image_center: The center of the rotation of the image.
        scale_factor: Scaling factor for matching template size to image size

    Returns:
        A tuple containing:
            - The refined location (x, y) in the rotated image coordinates.
            - The rotated initial location
            - The original initial location
        Returns (None, None, None) if template matching fails.
    """

    chip_type = user_chip_mapping.get('chip_type')
    rotation_angle = user_chip_mapping.get('rotation_angle', 0)
    scale_factor = user_chip_mapping.get('scale_factor', 1.0)
    image_center = (image.shape[1] / 2, image.shape[0] / 2)
    user_features = user_chip_mapping.get('features', None)

    if not user_features:
        return None

    for idx, f in enumerate(user_features):
        label = f.get('label')
        user_location = f.get('user_location')

        # Rotate the user location to match the rotated image
        rotated_user_location = rotate_user_feature_locations(user_location, image_center, rotation_angle)

        # Load template image
        template = get_template_image_from_label(chip_type, label)

        # Scale template to match image size
        template = scale_template(template, scale_factor)

        # Define a search window around the rotated initial location
        search_window_size = (template.shape[1] * 3, template.shape[0] * 3)
        x_start = max(0, int(rotated_user_location[0] - search_window_size[0] / 2))
        y_start = max(0, int(rotated_user_location[1] - search_window_size[1] / 2))
        x_end = min(image.shape[1], int(rotated_user_location[0] + search_window_size[0] / 2))
        y_end = min(image.shape[0], int(rotated_user_location[1] + search_window_size[1] / 2))

        search_window = image[y_start:y_end, x_start:x_end]

        # Perform template matching
        result = cv2.matchTemplate(search_window, template, cv2.TM_CCORR_NORMED)

        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

        if max_val > 0.7:
            refined_x = x_start + max_loc[0]
            refined_y = y_start + max_loc[1]
            user_chip_mapping['features'][idx]['refined_location'] = [refined_x, refined_y]
        else:
            user_chip_mapping['features'][idx]['refined_location'] = None

    return user_chip_mapping, [refined_x, refined_y]

In [112]:
"""Main function to perform image alignment and feature localization."""
# 1. Load data
feature_location_json = Path('../config/FeatureLocation.json')
chip_location_json = Path('../Label_templates/Chip_map_list.json')

image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
image = cv2.normalize(
        image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U # type: ignore
    )  # Normalize to 8-bit range (0-255)
if len(image.shape) > 2:  # Check if the image has more than one channel (i.e., it's not already greyscale)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if image is None:
    print(f"Error: Could not load image from {image_path}")
    exit(1)

user_chip_mapping = load_user_feature_locations(feature_location_json)
user_chip_mapping = load_chip_feature_locations(chip_location_json, user_chip_mapping)
user_chip_mapping, rotation_angle = user_chip_rotation_angle(user_chip_mapping)
user_chip_mapping, scale_factor = user_chip_scale_factor(user_chip_mapping)

# 2. Rotate image using user angle
rotated_image = rotate_image(image, -rotation_angle)
if rotated_image is None:
    print("Error: Image rotation and scaling failed.")
    exit(1)

# 3. Refine feature locations using template matching
user_chip_mapping, refined_locations = refine_feature_locations(rotated_image, user_chip_mapping)

# 4. Refine rotation angle now we've located the image features
refined_f1 = user_chip_mapping['features'][0]['refined_location']
refined_f2 = user_chip_mapping['features'][1]['refined_location']
refined_angle = angle_between_points(refined_f1, refined_f2)

chip_f1 = user_chip_mapping['features'][0]['chip_location']
chip_f2 = user_chip_mapping['features'][1]['chip_location']
map_angle = angle_between_points(chip_f1, chip_f2)

refined_rotation_angle = map_angle - refined_angle
user_chip_mapping['rotation_angle'] = rotation_angle + refined_rotation_angle

# 5. Rotate image by refined rotation angle
rotated_image = rotate_image(image, -user_chip_mapping['rotation_angle'])
if rotated_image is None:
    print("Error: Image rotation and scaling failed.")
    exit(1)

# 6. Refine feature locations again with new rotation angle using template matching
user_chip_mapping, refined_locations = refine_feature_locations(rotated_image, user_chip_mapping)

# 6. Display results
print(f'User chip mapping: {user_chip_mapping}')

# Draw circles at the refined locations
user_features = user_chip_mapping.get('features', None)
for f in user_features:
    label = f.get('label')
    user_location = f.get('user_location')
    cv2.circle(rotated_image, user_location, 5, (255, 255, 255), 2)
    chip_location = f.get('chip_location')
    cv2.circle(rotated_image, chip_location, 25, (255, 255, 255), 2)
    refined_location = f.get('refined_location')
    cv2.circle(rotated_image, refined_location, 10, (255, 255, 255), 2)
cv2.imshow('Rotated Image with Refined Locations', rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

User chip mapping: {'chip_type': 'IMECII_2', 'features': [{'label': 'D-Right', 'user_location': [1840, 849], 'chip_location': [2261, -1040], 'refined_location': [1821, 831]}, {'label': 'E-Right', 'user_location': [1848, 122], 'chip_location': [2261, -2040], 'refined_location': [1821, 106]}], 'user_location_angle': -89.36953543860086, 'chip_location_angle': -90.0, 'rotation_angle': -0.23587176756903716, 'scale_factor': 0.7270440151737719}
